In [1]:
import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
import numpy as np
import scipy
from tqdm import tqdm
import bilby
from jax.config import config; config.update("jax_enable_x64", True)


from sklearn import mixture

from corner import corner, hist2d

import jaxopt

from tqdm import tqdm

import matplotlib.pyplot as plt

import pystroke

/home/ethan.payne/.conda/envs/ve310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/local/ethan.payne/ipykernel_1387763/2116840790.py:9: DeprecationWarning: Accessing jax.config via the jax.config submodule is deprecated.
  from jax.config import config; config.update("jax_enable_x64", True)


In [2]:
def uniform_generator(minimum, maximum):
    
    def uniform(x):
        return (x - minimum)/(maximum - minimum)
    
    return uniform

prior_cdfs = [uniform_generator(-10,10), uniform_generator(-10,10)]

In [3]:
dims = 2
Nobs = 30

# Construct random posteriors
event_list = []
pdet = dist.Uniform(-5*jnp.ones(dims),5*jnp.ones(dims))

means = []
covs = []

for i in tqdm(range(Nobs)):
    
    theta = np.random.uniform(0, 2*np.pi)
    
    cov = np.dot(
        np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]]).T,
        np.dot(
        np.diag(np.random.uniform(0.01,0.5,size=dims)),
        np.array([[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]])
    ))
    
    loc = np.random.randn(dims) + np.random.multivariate_normal(np.zeros(dims), cov)
    
    means.append(loc)
    covs.append(cov)
    
    key = jax.random.PRNGKey(np.random.randint(0,100000))
    dist_i_samples = dist.MultivariateNormal(jnp.array(loc), jnp.array(cov)).sample(key, sample_shape=(2000,))
    
    event_list.append(pystroke.GMMDistribution(dist_i_samples, prior_cdfs))

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
100%|██████████| 30/30 [00:13<00:00,  2.29it/s]


In [4]:
xs = np.linspace(-3,3,100)
ys = np.linspace(-3,3,100)
zs = np.zeros(100)

for i in range(Nobs):
    
    plt.plot(xs, np.exp(event_list[i].log_prob(jnp.array([xs,xs]).T)))
    
plt.show()

In [5]:
pistroke_test = pystroke.PiStroke(event_list, pdet)

In [6]:
_ = pistroke_test.gradient_descent(iterations=2, tol=np.array([1e-4,1e-2]))

  0%|          | 0/2 [00:00<?, ?it/s]

Random initialized position has logL -16.554621555169653


: 

In [7]:
xs = np.linspace(-3,3,100)
ys = np.linspace(-3,3,100)
zs = np.zeros(100)

for i in range(Nobs):
    
    plt.plot(xs, np.exp(event_list[i].log_prob(jnp.array([xs]).T))/5)
    
plt.scatter(pistroke_test.result_array_gd[:,0], 
            np.exp(pistroke_test.result_array_gd[:,-1])/np.sum(np.exp(pistroke_test.result_array_gd[:,-1])),
            zorder=100, color='r', 
            alpha=np.exp(pistroke_test.result_array_gd[:,-1])/np.max(np.exp(pistroke_test.result_array_gd[:,-1])))

for i in range(len(pistroke_test.result_array_gd)):
    plt.axvline(pistroke_test.result_array_gd[i,0])

plt.show()

In [8]:
fig = plt.figure(figsize=[6,6])

for i in range(len(event_list)):
    samples = event_list[i].samples
    hist2d(samples[:,0], samples[:,1], new_fig=False, levels=[0.9], 
           smooth=0.9, bins=25, plot_density=False, plot_datapoints=False, color='k', contour_kwargs={'alpha':0.1})

plt.scatter(pistroke_test.result_array_gd[:,0], pistroke_test.result_array_gd[:,1],zorder=100, color='r',
            alpha=np.exp(pistroke_test.result_array_gd[:,-1])/np.max(np.exp(pistroke_test.result_array_gd[:,-1])))

ax = plt.gca()
ax.set_aspect('equal')
ax.set_ylim(-4,4)
ax.set_xlim(-4,4)

(-4.0, 4.0)

In [9]:
np.exp(pistroke_test.result_array_gd[:,-1])

array([1.90567645e-01, 3.24639149e-01, 1.28286415e-01, 6.21738227e-02,
       5.93573451e-02, 5.81720398e-02, 1.67334143e-01, 5.02462344e-02,
       4.95650779e-02, 7.02566596e-02, 4.22955201e-02, 5.14512566e-02,
       1.35935542e-02, 1.07596809e-03, 1.38574292e-05])

In [10]:
pystroke.generate_O3_GMMs('/home/jacob.golomb/O3b/nov24/o1o2o3_default/init/result/o1o2o3_mass_c_iid_mag_iid_tilt_powerlaw_redshift_result.json')

13:06 bilby INFO    : Loading posteriors...
13:06 bilby INFO    : Loaded Overall from /home/ethan.payne/projects/evidencemaximizedprior/observing_run_PE/O1O2/GW151012_GWTC-1.hdf5.
13:06 bilby INFO    : Loaded Overall from /home/ethan.payne/projects/evidencemaximizedprior/observing_run_PE/O1O2/GW170823_GWTC-1.hdf5.
13:06 bilby INFO    : Loaded Overall from /home/ethan.payne/projects/evidencemaximizedprior/observing_run_PE/O1O2/GW151226_GWTC-1.hdf5.
13:06 bilby INFO    : Loaded Overall from /home/ethan.payne/projects/evidencemaximizedprior/observing_run_PE/O1O2/GW150914_GWTC-1.hdf5.
13:06 bilby INFO    : Loaded Overall from /home/ethan.payne/projects/evidencemaximizedprior/observing_run_PE/O1O2/GW170104_GWTC-1.hdf5.
13:06 bilby INFO    : Loaded Overall from /home/ethan.payne/projects/evidencemaximizedprior/observing_run_PE/O1O2/GW170818_GWTC-1.hdf5.
13:06 bilby INFO    : Loaded Overall from /home/ethan.payne/projects/evidencemaximizedprior/observing_run_PE/O1O2/GW170729_GWTC-1.hdf5.
13:0

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5,) + inhomogeneous part.

In [ ]:

from gwpopulation_pipe.data_collection import load_all_events

In [ ]:
load_all_events?